# Creating, Evaluating, and Deploying a Fraud Detection Model

## Introduction

In this notebook, we'll demonstrate data engineering and data science work flow with an e2e sample. The scenario is to build a model for detecting fraud credit card transactions.

## Step 1: Load the Data

The dataset contains transactions made by credit cards in September 2013 by European cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependent cost-sensitive learning. Feature 'Class' is the response variable, and it takes value 1 in case of fraud and 0 otherwise.

Given the class imbalance ratio, we recommend measuring the accuracy using the Area Under the Precision-Recall Curve (AUPRC). Confusion matrix accuracy is not meaningful for unbalanced classification.

- creditcard.csv

|"Time"|"V1"|"V2"|"V3"|"V4"|"V5"|"V6"|"V7"|"V8"|"V9"|"V10"|"V11"|"V12"|"V13"|"V14"|"V15"|"V16"|"V17"|"V18"|"V19"|"V20"|"V21"|"V22"|"V23"|"V24"|"V25"|"V26"|"V27"|"V28"|"Amount"|"Class"|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|0|-1.3598071336738|-0.0727811733098497|2.53634673796914|1.37815522427443|-0.338320769942518|0.462387777762292|0.239598554061257|0.0986979012610507|0.363786969611213|0.0907941719789316|-0.551599533260813|-0.617800855762348|-0.991389847235408|-0.311169353699879|1.46817697209427|-0.470400525259478|0.207971241929242|0.0257905801985591|0.403992960255733|0.251412098239705|-0.018306777944153|0.277837575558899|-0.110473910188767|0.0669280749146731|0.128539358273528|-0.189114843888824|0.133558376740387|-0.0210530534538215|149.62|"0"|
|0|1.19185711131486|0.26615071205963|0.16648011335321|0.448154078460911|0.0600176492822243|-0.0823608088155687|-0.0788029833323113|0.0851016549148104|-0.255425128109186|-0.166974414004614|1.61272666105479|1.06523531137287|0.48909501589608|-0.143772296441519|0.635558093258208|0.463917041022171|-0.114804663102346|-0.183361270123994|-0.145783041325259|-0.0690831352230203|-0.225775248033138|-0.638671952771851|0.101288021253234|-0.339846475529127|0.167170404418143|0.125894532368176|-0.00898309914322813|0.0147241691924927|2.69|"0"|

### Install libraries
In this notebook, we'll use `imblearn` which first needs to be installed. The PySpark kernel will be restarted after `%pip install`, thus we need to install it before we run any other cells.

In [ ]:
# install imblearn for SMOTE
%pip install imblearn

**By defining below parameters, we can apply this notebook on different datasets easily.**

In [ ]:
IS_CUSTOM_DATA = False  # if True, dataset has to be uploaded manually

TARGET_COL = "Class"  # target column name
IS_SAMPLE = False  # if True, use only <SAMPLE_ROWS> rows of data for training, otherwise use all data
SAMPLE_ROWS = 5000  # if IS_SAMPLE is True, use only this number of rows for training

DATA_FOLDER = "Files/fraud-detection/"  # folder with data files
DATA_FILE = "creditcard.csv"  # data file name

EXPERIMENT_NAME = "aisample-fraud"  # mlflow experiment name

### Download dataset and Upload to lakehouse

**Please add a lakehouse to the notebook before running it.**

In [ ]:
if not IS_CUSTOM_DATA:
    # Download demo data files into lakehouse if not exist
    import os, requests

    remote_url = "https://synapseaisolutionsa.blob.core.windows.net/public/Credit_Card_Fraud_Detection"
    fname = "creditcard.csv"
    download_path = f"/lakehouse/default/{DATA_FOLDER}/raw"

    if not os.path.exists("/lakehouse/default"):
        raise FileNotFoundError(
            "Default lakehouse not found, please add a lakehouse and restart the session."
        )
    os.makedirs(download_path, exist_ok=True)
    if not os.path.exists(f"{download_path}/{fname}"):
        r = requests.get(f"{remote_url}/{fname}", timeout=30)
        with open(f"{download_path}/{fname}", "wb") as f:
            f.write(r.content)
    print("Downloaded demo data files into lakehouse.")

In [ ]:
# to record the notebook running time
import time

ts = time.time()

### Read data from lakehouse

In [ ]:
df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", True)
    .load(f"{DATA_FOLDER}/raw/{DATA_FILE}")
    .cache()
)

## Step 2. Exploratory Data Analysis

### Display Raw Data

We can explore the raw data with `display`, do some basic statistics or even show chart views.

In [ ]:
display(df)

In [ ]:
# print dataset basic info
print("records read: " + str(df.count()))
print("Schema: ")
df.printSchema()

### Cast columns into the correct types

In [ ]:
import pyspark.sql.functions as F

df_columns = df.columns
df_columns.remove(TARGET_COL)

# to make sure the TARGET_COL is the last column
df = df.select(df_columns + [TARGET_COL]).withColumn(
    TARGET_COL, F.col(TARGET_COL).cast("int")
)

if IS_SAMPLE:
    df = df.limit(SAMPLE_ROWS)

## Step 3. Model development and deploy
So far, we have explored the dataset, checked the scheme, adjusted the columns order, and casted the columns into correct types.

Next, we'll train a lightgbm model to classify fraud transactions.

### Prepare training and testing data

In [ ]:
# Split the dataset into train and test
from sklearn.model_selection import train_test_split

df_pd = df.toPandas()
train, test = train_test_split(df_pd, test_size=0.15)
feature_cols = [c for c in df_pd.columns.tolist() if c not in [TARGET_COL]]

### Check data volume and imbalance

In [ ]:
train.groupby(TARGET_COL).size()

### Handle imbalance data
We'll apply [SMOTE](https://arxiv.org/abs/1106.1813) (Synthetic Minority Over-sampling Technique) to automatically handle imbalance data. A dataset is imbalanced if the classification categories are not approximately equally represented. Often real-world data sets are predominately composed of "normal" examples with only a small percentage of "abnormal" or "interesting" examples. It is also the case that the cost of misclassifying an abnormal (interesting) example as a normal example is often much higher than the cost of the reverse error. Under-sampling of the majority (normal) class has been proposed as a good means of increasing the sensitivity of a classifier to the minority class. This paper shows that a combination of our method of over-sampling the minority (abnormal) class and under-sampling the majority (normal) class can achieve better classifier performance (in ROC space) than only under-sampling the majority class. 

#### Apply SMOTE for new train_data 
imblearn only works for pandas dataframe, not pyspark dataframe.

In [ ]:
import pandas as pd
from collections import Counter
from imblearn.over_sampling import SMOTE

X = train[feature_cols]
y = train[TARGET_COL]
print("Original dataset shape %s" % Counter(y))

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)
print("Resampled dataset shape %s" % Counter(y_res))

new_train = pd.concat([X_res, y_res], axis=1)

### Define the Model

With our data in place, we can now define the model. We'll apply lightgbm model in this notebook. 

We'll leverage SynapseML to implement the model within a few lines of code.

In [ ]:
import lightgbm as lgb

model = lgb.LGBMClassifier(objective="binary")
smote_model = lgb.LGBMClassifier(objective="binary")

### Model training

In [ ]:
print("Start training with raw data:\n")
model = model.fit(
    train[feature_cols],
    train[TARGET_COL],
    eval_set=[(test[feature_cols], test[TARGET_COL])],
    eval_metric="auc",
    callbacks=[
        lgb.log_evaluation(10),
    ],
)

print(f"\n\nStart training with SMOTE enhanced data:\n")
smote_model = smote_model.fit(
    new_train[feature_cols],
    new_train[TARGET_COL],
    eval_set=[(test[feature_cols], test[TARGET_COL])],
    eval_metric="auc",
    callbacks=[
        lgb.log_evaluation(10),
    ],
)

### Model Explanation
Here we can show the importance of each column.

In [ ]:
lgb.plot_importance(model, title="Feature importance for imbalanced raw data")
lgb.plot_importance(smote_model, title="Feature importance for SMOTE enhanced data")

### Model Evaluation

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType


def prediction_to_spark(model, test):
    predictions = model.predict(test[feature_cols], num_iteration=model.best_iteration_)
    predictions = tuple(zip(test[TARGET_COL].tolist(), predictions.tolist()))
    dataColumns = [TARGET_COL, "prediction"]
    predictions = (
        spark.createDataFrame(data=predictions, schema=dataColumns)
        .withColumn(TARGET_COL, col(TARGET_COL).cast(IntegerType()))
        .withColumn("prediction", col("prediction").cast(DoubleType()))
    )

    return predictions

In [ ]:
predictions = prediction_to_spark(model, test)
predictions.limit(10).toPandas()

In [ ]:
from synapse.ml.train import ComputeModelStatistics

metrics = ComputeModelStatistics(
    evaluationMetric="classification", labelCol=TARGET_COL, scoredLabelsCol="prediction"
).transform(predictions)
display(metrics)

In [ ]:
# collect confusion matrix value
cm = metrics.select("confusion_matrix").collect()[0][0].toArray()
print(cm)

In [ ]:
# plot confusion matrix
import seaborn as sns

sns.set(rc={"figure.figsize": (6, 4.5)})
ax = sns.heatmap(cm, annot=True, fmt=".20g")
ax.set_title("Confusion Matrix")
ax.set_xlabel("Predicted label")
ax.set_ylabel("True label")

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator


def evaluate(predictions):
    """
    Evaluate the model by computing AUROC and AUPRC with the predictions.
    """

    # initialize the binary evaluator
    evaluator = BinaryClassificationEvaluator(
        rawPredictionCol="prediction", labelCol=TARGET_COL
    )

    _evaluator = lambda metric: evaluator.setMetricName(metric).evaluate(predictions)

    # calculate AUROC, baseline 0.5
    auroc = _evaluator("areaUnderROC")
    print(f"AUROC: {auroc:.4f}")

    # calculate AUPRC, baseline positive rate (0.172% in the demo data)
    auprc = _evaluator("areaUnderPR")
    print(f"AUPRC: {auprc:.4f}")

    return auroc, auprc

In [ ]:
# evaluate the original model
auroc, auprc = evaluate(predictions)

In [ ]:
# evaluate the SMOTE model
new_predictions = prediction_to_spark(smote_model, test)
new_auroc, new_auprc = evaluate(new_predictions)

### Log Model with MLFlow
Now we can save the trained models for later use. Here we use mlflow to log metrics/models.

In [ ]:
# setup mlflow
import mlflow
from mlflow.models.signature import infer_signature
import numpy as np

signature = infer_signature(
    test[feature_cols],
    np.array(
        model.predict(test[feature_cols], num_iteration=model.best_iteration_),
        dtype="int_",
    ),
)

mlflow.set_experiment(EXPERIMENT_NAME)

In [ ]:
# log model, metrics and params
raw_model_info = [model, auprc, auroc]
smote_model_info = [smote_model, new_auprc, new_auroc]
data_enhancements = ["None", "SMOTE"]

for idx, info in enumerate([raw_model_info, smote_model_info]):
    model, auprc, auroc = info
    with mlflow.start_run() as run:
        print("log model:")
        mlflow.lightgbm.log_model(
            model,
            f"{EXPERIMENT_NAME}-lightgbm",
            registered_model_name=f"{EXPERIMENT_NAME}-lightgbm",
            signature=signature,
        )

        print("log metrics:")
        mlflow.log_metrics({"AUPRC": auprc, "AUROC": auroc})

        print("log parameters:")
        mlflow.log_params(
            {"Data_Enhancement": data_enhancements[idx], "DATA_FILE": DATA_FILE}
        )

## Step 4. Save Prediction Results

### Prediction with *Predict* Transformer

In [ ]:
from synapse.ml.predict import MLFlowTransformer

spark.conf.set("spark.synapse.ml.predict.enabled", "true")

model = MLFlowTransformer(
    inputCols=feature_cols,
    outputCol="prediction",
    modelName=f"{EXPERIMENT_NAME}-lightgbm",
    modelVersion=1,
)

test_spark = spark.createDataFrame(data=test, schema=test.columns.to_list())

batch_predictions = model.transform(test_spark)

In [ ]:
# code for saving predictions into lakehouse
batch_predictions.write.format("delta").mode("overwrite").save(
    f"{DATA_FOLDER}/predictions/batch_predictions"
)

In [ ]:
batch_predictions.limit(5).toPandas()

In [ ]:
print(f"Full run cost {int(time.time() - ts)} seconds.")